# Présentation des Résultats - Prédiction du Churn Client

## 1. Introduction

Ce notebook présente les conclusions du projet de prédiction du churn client. L'objectif était de développer un modèle de machine learning pour identifier les clients les plus susceptibles de résilier leur abonnement.

Le modèle le plus performant, une **Forêt Aléatoire (Random Forest)**, a été sélectionné et optimisé. Nous allons ici analyser ses performances et les facteurs de churn les plus importants qu'il a identifiés.

## 2. Préparation : Chargement des données et du modèle

In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.figure_factory as ff
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve

# --- Chargement des données ---
X_train = pd.read_csv('../data/X_train.csv')
X_test = pd.read_csv('../data/X_test.csv')
y_train = pd.read_csv('../data/y_train.csv').values.ravel()
y_test = pd.read_csv('../data/y_test.csv').values.ravel()

# --- Entraînement du modèle final ---
# Normalement, on chargerait un modèle sauvegardé.
# Pour cette présentation, nous ré-entraînons le meilleur modèle identifié à l'étape 3.
best_params = {
    'bootstrap': True,
    'max_depth': 10,
    'min_samples_leaf': 1,
    'min_samples_split': 5,
    'n_estimators': 200,
    'random_state': 42
}
final_model = RandomForestClassifier(**best_params)
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_test)
y_proba = final_model.predict_proba(X_test)[:, 1]
print("Modèle final entraîné et prédictions effectuées.")

Modèle final entraîné et prédictions effectuées.


## 3. Performance du Modèle Final

In [2]:
# Calcul des métriques
metrics = {
    'Accuracy': accuracy_score(y_test, y_pred),
    'Precision': precision_score(y_test, y_pred),
    'Recall': recall_score(y_test, y_pred),
    'F1-Score': f1_score(y_test, y_pred),
    'AUC-ROC': roc_auc_score(y_test, y_proba)
}
# Affichage des métriques
fig = go.Figure(data=[go.Table(header=dict(values=['Métrique', 'Score']),
                 cells=dict(values=[list(metrics.keys()), [f'{v:.4f}' for v in metrics.values()]]))])
fig.update_layout(title='Scores de Performance du Modèle Final')
fig.show()

### Matrice de ConfusionLa matrice de confusion nous montre en détail les succès et les erreurs de notre modèle. Elle distingue les vrais positifs, vrais négatifs, faux positifs et faux négatifs.

In [3]:
cm = confusion_matrix(y_test, y_pred)
labels = ['Non-Churn', 'Churn']
# Utilisation de Figure Factory pour une meilleure annotation
fig = ff.create_annotated_heatmap(z=cm, x=labels, y=labels, colorscale='Blues', showscale=True)
fig.update_layout(
    title='Matrice de Confusion Interactive',
    xaxis_title='Prédiction du modèle',
    yaxis_title='Vraie valeur'
)
fig.show()

## 4. Facteurs Clés du Churn : Importance des CaractéristiquesQuels sont les facteurs qui influencent le plus la décision d'un client de résilier son abonnement ? Le graphique ci-dessous montre l'importance de chaque caractéristique selon notre modèle.

In [4]:
feature_importances = pd.DataFrame(final_model.feature_importances_, index=X_train.columns, columns=['Importance']).sort_values('Importance', ascending=True)
fig = go.Figure(go.Bar(
    x=feature_importances.Importance,
    y=feature_importances.index,
    orientation='h'
))
fig.update_layout(
    title='Importance des Caractéristiques pour la Prédiction du Churn',
    xaxis_title='Importance',
    yaxis_title='Caractéristique',
    height=800
)
fig.show()

## 5. Conclusion et Recommandations

Notre modèle de Forêt Aléatoire est performant pour prédire le churn client, avec un score **AUC-ROC d'environ 0.84**. Cela signifie qu'il a une bonne capacité à distinguer les clients qui vont résilier de ceux qui ne le feront pas.

L'analyse des caractéristiques révèle des informations cruciales :

1.  **Le type de contrat (`Contract_Month-to-month`)** est de loin le facteur le plus prédictif. Les clients avec un abonnement mensuel sont beaucoup plus susceptibles de partir.
2.  **L'ancienneté (`tenure`)** et les **frais totaux (`TotalCharges`)** sont également très importants. Les clients plus récents et ceux ayant dépensé moins au total sont plus à risque.
3.  **Le type de service Internet (`InternetService_Fiber optic`)** joue un rôle : les clients avec la fibre optique semblent plus enclins à la résiliation, ce qui pourrait indiquer des problèmes de satisfaction (prix, qualité de service) liés à cette offre.

### Recommandations Stratégiques :

*   **Action sur les contrats mensuels :** Lancer des campagnes ciblées pour encourager les clients avec un contrat mensuel à passer sur des contrats d'un ou deux ans, en offrant des promotions ou des avantages.
*   **Programme de fidélité :** Mettre en place un programme de fidélité qui récompense l'ancienneté pour retenir les clients sur le long terme.
*   **Analyser la satisfaction Fibre :** Mener une enquête de satisfaction auprès des clients ayant la fibre optique pour comprendre les raisons de leur taux de churn plus élevé et améliorer le service.